In [1]:
import torchaudio
import os
import torch
from utils import InitializationTrain
CONSTANTS = InitializationTrain()

In [3]:
waveform, sample_rate = torchaudio.load("/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S08508_L0201_E018.wav")

In [2]:
waveform, sample_rate = torchaudio.load("/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S08508_L0201_E018_mono_1.wav")
print(waveform.shape)

torch.Size([1, 183680])


In [3]:
waveform = waveform[0, 2 * sample_rate: -sample_rate]

In [4]:
resampler = torchaudio.transforms.Resample(
            orig_freq=CONSTANTS.orig_freq, new_freq=CONSTANTS.new_freq
        )
speech_l, sr = torchaudio.load("/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S08508_L0201_E018_mono_1.wav")
speech_l = torch.squeeze(speech_l)
speech_l = speech_l[2 * sr: -sr]
if sr != CONSTANTS.new_freq:
            speech_l = resampler(speech_l)
            sr = CONSTANTS.new_freq
            
desired_length = sr * 6  # keep 6 seconds
if speech_l.size(-1) < desired_length:
    padding = desired_length - speech_l.size(-1)
    speech_l = torch.nn.functional.pad(speech_l, (0, padding), "constant")
elif speech_l.size(-1) > desired_length:
    speech_l = speech_l[..., :desired_length]



In [5]:
speech_l.shape

torch.Size([96000])

In [6]:
waveform.shape

torch.Size([87680])

In [4]:
len(waveform)

2

In [5]:
print(waveform)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [6]:
waveform_1 = waveform[0,:]

In [7]:
waveform_1.shape

torch.Size([183680])

In [12]:
waveform.mean(dim=0, keepdim=True).shape

torch.Size([1, 183680])

In [13]:
torchaudio.save(filepath="./123.wav", src=waveform_1.reshape(1, -1), sample_rate=sample_rate)

In [6]:
import os
import torch
import datetime
import time

import torch

import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import subprocess
import os
from utils import *
from models import *
from my_loss import *
from data_process import *

[NeMo W 2023-04-22 01:47:21 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-22 01:47:22 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2023-04-22 01:47:27 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [7]:
dataset = CPCdataBinaural(metadata=CONSTANTS.metadata)

In [9]:
for i, j in dataset:
    print(i)
    print(len(i))
    print(j)
    break

[tensor([-0.1319, -0.0580, -0.2344,  ...,  0.0000,  0.0000,  0.0000]), tensor([-0.0159,  0.0873,  0.0218,  ...,  0.0000,  0.0000,  0.0000])]
2
OrderedDict([('path', ['/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S08510_L0239_E001_mono_1.wav', '/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S08510_L0239_E001_mono_2.wav']), ('score', 0.1), ('listener', 'L0239'), ('system', 'E001'), ('scene', 'S08510'), ('volume', 0.56), ('prompt', "i suppose you wouldn't be free for dinner this evening")])


In [12]:
train_loader = DataLoader(dataset=dataset, batch_size=3)

In [18]:
for i, j in train_loader:
    print(i[0].shape)
    print(i)
    print(len(i))
    print(i[1].shape)
    break

torch.Size([3, 96000])
[tensor([[-0.1319, -0.0580, -0.2344,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0542,  0.0237, -0.0008,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0822, -0.0206,  0.2036,  ...,  0.0000,  0.0000,  0.0000]]), tensor([[-0.0159,  0.0873,  0.0218,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2010, -0.2137,  0.1628,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1564, -0.4136, -0.3850,  ...,  0.0000,  0.0000,  0.0000]])]
2
torch.Size([3, 96000])
